# Calculate the distance matrix for anisotropic geospatial correlation. 

Author: Yaolin Ge
Email: geyaolin@gmail.com
Date: 2023-05-26

### Methodology: 
1. Calculate the distance difference between each pair of points in x, y, z axis. 
2. Scale the z axis up by a factor of constant which is determined by the radio between the horizontal and vertical correlation length.
3. Calculate the distance matrix using the scaled distance difference.

In [2]:
import numpy as np
from scipy.spatial.distance import cdist

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
N = 25
xv = np.linspace(0, 1, N)
yv = np.linspace(0, 1, N)
zv = np.linspace(0, 1, N)

xx, yy, zz = np.meshgrid(xv, yv, zv, indexing='ij')

x = xx.flatten()
y = yy.flatten()
z = zz.flatten()
grid = np.vstack((x, y, z)).T

ksi = 10

In [7]:
# Method 1: calculate distance matrix manually
def cal_distance_matrix(grid1, grid2, ksi): 
    dx = grid1[:, 0].reshape(-1, 1) @ np.ones((1, grid2.shape[0])) - np.ones((grid1.shape[0], 1)) @ grid2[:, 0].reshape(1, -1)
    dy = grid1[:, 1].reshape(-1, 1) @ np.ones((1, grid2.shape[0])) - np.ones((grid1.shape[0], 1)) @ grid2[:, 1].reshape(1, -1)
    dz = grid1[:, 2].reshape(-1, 1) @ np.ones((1, grid2.shape[0])) - np.ones((grid1.shape[0], 1)) @ grid2[:, 2].reshape(1, -1)
    d = np.sqrt(dx**2 + dy**2 + (ksi * dz)**2)
    return d 

# %timeit cal_distance_matrix(grid, grid, ksi)

# dm1 = cal_distance_matrix(grid, grid, ksi)
loc1 = np.array([[0, 0, 0]])
loc2 = np.array([[1, 1, 1]])

cal_distance_matrix(loc1, loc2, ksi)

dx:  [[-1.]]
dy:  [[-1.]]
dz:  [[-1.]]
d:  [[10.09950494]]


array([[10.09950494]])

In [9]:
np.sqrt(102)

10.099504938362077

In [5]:
# Method 2, use scipy.spatial.distance.cdist with metric "user_defined"
def metric(loc1, loc2, ksi=ksi):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    dz = loc1[2] - loc2[2]
    d = np.sqrt(dx**2 + dy**2 + (ksi * dz)**2)
    return d

%timeit cdist(grid, grid, metric=metric)
dm2 = cdist(grid, grid, metric=metric)

KeyboardInterrupt: 

In [17]:
np.all(dm1 == dm2)

True